# Limpieza de datos

## Imports y funciones a usar

In [15]:
import numpy as np
import funciones as fn
import math

In [2]:
def datos_boxplot(atributo):
    Q1 = df[atributo].quantile(0.25)
    Q3 = df[atributo].quantile(0.75)
    Q2 = df[atributo].quantile(0.50)
    minimo = df[atributo].min()
    maximo = df[atributo].max()
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR if (Q1 - 1.5 * IQR) > 0 else 0
    limite_superior = Q3 + 1.5 * IQR
    return {"q1": Q1, "q2": Q2, "q3": Q3, "iqr": IQR, "limite_inferior": limite_inferior, "limite_superior": limite_superior, "minimo": minimo, "maximo": maximo}

In [3]:
def round_away_from_zero_int(n):
    return math.floor(n + 0.5) if n >= 0 else math.ceil(n - 0.5)

In [16]:
df = fn.cargar_datos()
inicial = df.shape
df.shape

(39644, 61)

## Limpieza vertical (Articulos)

### Datos boxplot del atributo "n_tokens_content" (cantidad de tokens en el contenido del articulo) y n_tokens_title (cantidad de tokens en el titulo del articulo)

#### n_tokens_content - cantidad de tokens en el contenido del articulo

In [17]:
#Q1, Q2, Q3, IQR, limite_inferior, limite_superior, minimo, maximo = datos_boxplot("n_tokens_content")
dict_resultado_tc = datos_boxplot("n_tokens_content")
print(f"Q1: {dict_resultado_tc["q1"]},"
      f"Q2: {dict_resultado_tc["q2"]},"
      f"Q3: {dict_resultado_tc["q3"]}, "
      f"IQR: {dict_resultado_tc["iqr"]}")
print(f"Limite inferior: {dict_resultado_tc["limite_inferior"]}, "
      f"Limite superior: {dict_resultado_tc["limite_superior"]}")
print(f"Minimo: {dict_resultado_tc["minimo"]}, Maximo: {dict_resultado_tc["maximo"]}")

Q1: 246.0,Q2: 409.0,Q3: 716.0, IQR: 470.0
Limite inferior: 0, Limite superior: 1421.0
Minimo: 0.0, Maximo: 8474.0


El limite inferior se ajusta a 0 ya que no puede haber valores negativos en la cantidad de tokens.<br>
Vemos que hay artuculos con 0 tokens en el contenido.

#### n_tokens_title - cantidad de tokens en el titulo del articulo

In [18]:
dict_resultado_tt = datos_boxplot("n_tokens_title")
print(f"Q1: {dict_resultado_tt["q1"]}, "
      f"Q2: {dict_resultado_tt["q2"]}, "
      f"Q3: {dict_resultado_tt["q3"]}, "
      f"IQR: {dict_resultado_tt["iqr"]}")
print(f"Limite inferior: {dict_resultado_tt["limite_inferior"]}, "
      f"Limite superior: {dict_resultado_tt["limite_superior"]}")
print(f"Minimo: {dict_resultado_tt["minimo"]}, "
      f"Maximo: {dict_resultado_tt["maximo"]}")

Q1: 9.0, Q2: 10.0, Q3: 12.0, IQR: 3.0
Limite inferior: 4.5, Limite superior: 16.5
Minimo: 2.0, Maximo: 23.0


Aqui vemos que no hay titulos con 0 tokens.

### Limpieza de articulos en base a los calculos anteriores

#### Limpieza en base a n_tokens_content

In [19]:
antes = df.shape[0]
atributo = "n_tokens_content"
mask = (df[atributo] > dict_resultado_tc["limite_inferior"]) & (df[atributo] <= dict_resultado_tc["limite_superior"])
df = df[mask].reset_index(drop=True)
despues = df.shape[0]
print(f"Antes: {antes} articulos, Despues: {despues} articulos, Eliminados: {antes-despues} articulos")
print(f"Nuevo minimo de {atributo}: {df[atributo].min()} \n"
      f"Nuevo maximo de {atributo}: {df[atributo].max()}")

Antes: 39644 articulos, Despues: 36530 articulos, Eliminados: 3114 articulos
Nuevo minimo de n_tokens_content: 18.0 
Nuevo maximo de n_tokens_content: 1421.0


#### Limpieza en base a n_tokens_title

In [20]:
antes = df.shape[0]
atributo = "n_tokens_title"
mask = (df[atributo] <= dict_resultado_tt["limite_superior"])
df = df[mask].reset_index(drop=True)
despues = df.shape[0]
print(f"Antes: {antes} articulos, Despues: {despues} articulos, Eliminados: {antes-despues} articulos")
print(f"Nuevo minimo de {atributo}: {df[atributo].min()} \n"
      f"Nuevo maximo de {atributo}: {df[atributo].max()}")

Antes: 36530 articulos, Despues: 36423 articulos, Eliminados: 107 articulos
Nuevo minimo de n_tokens_title: 2.0 
Nuevo maximo de n_tokens_title: 16.0


#### Resultados finales de la limpieza vertical

In [21]:
print(f"Antes: {inicial[0]} articulos, Despues: {despues} articulos, Eliminados: {inicial[0]-despues} articulos, Porcentaje eliminados: {(inicial[0]-despues)/inicial[0]*100:.2f}%")

Antes: 39644 articulos, Despues: 36423 articulos, Eliminados: 3221 articulos, Porcentaje eliminados: 8.12%


# Limpieza horizontal - Atributos

In [22]:
dic_atributos = fn.diccionario_tipos_atributos()
atributos_a_eliminar = np.concatenate([dic_atributos['atributos_min_max_avg_terceros'], ["url", "timedelta"]])
print(atributos_a_eliminar)

['kw_min_min' 'kw_max_min' 'kw_avg_min' 'kw_min_max' 'kw_max_max'
 'kw_avg_max' 'kw_min_avg' 'kw_max_avg' 'kw_avg_avg'
 'self_reference_min_shares' 'self_reference_max_shares'
 'self_reference_avg_sharess' 'url' 'timedelta']


In [23]:

df.drop(atributos_a_eliminar, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36423 entries, 0 to 36422
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   n_tokens_title                 36423 non-null  float64
 1   n_tokens_content               36423 non-null  float64
 2   n_unique_tokens                36423 non-null  float64
 3   n_non_stop_words               36423 non-null  float64
 4   n_non_stop_unique_tokens       36423 non-null  float64
 5   num_hrefs                      36423 non-null  float64
 6   num_self_hrefs                 36423 non-null  float64
 7   num_imgs                       36423 non-null  float64
 8   num_videos                     36423 non-null  float64
 9   average_token_length           36423 non-null  float64
 10  num_keywords                   36423 non-null  float64
 11  data_channel_is_lifestyle      36423 non-null  float64
 12  data_channel_is_entertainment  36423 non-null 

# Importacion a csv limpio

In [28]:
df.to_csv('../dataset/datos_limpios.csv', index=False)